In [9]:
from utils import filterPossible, getSplits, check
from wordLists import guesses, answers
from tqdm.auto import tqdm
from valuations import *
import numpy as np
import pandas as pd

In [4]:
G = sorted(guesses + answers)
S = sorted(answers)

In [5]:
len(G)

12947

In [16]:
C = list(S)
# C = filterPossible('trace', '00100' , C)
# C = filterPossible('bahus', '01000' , C)
C

['aback',
 'abase',
 'abate',
 'abbey',
 'abbot',
 'abhor',
 'abide',
 'abled',
 'abode',
 'abort',
 'about',
 'above',
 'abuse',
 'abyss',
 'acorn',
 'acrid',
 'actor',
 'acute',
 'adage',
 'adapt',
 'adept',
 'admin',
 'admit',
 'adobe',
 'adopt',
 'adore',
 'adorn',
 'adult',
 'affix',
 'afire',
 'afoot',
 'afoul',
 'after',
 'again',
 'agape',
 'agate',
 'agent',
 'agile',
 'aging',
 'aglow',
 'agony',
 'agree',
 'ahead',
 'aider',
 'aisle',
 'alarm',
 'album',
 'alert',
 'algae',
 'alibi',
 'alien',
 'align',
 'alike',
 'alive',
 'allay',
 'alley',
 'allot',
 'allow',
 'alloy',
 'aloft',
 'alone',
 'along',
 'aloof',
 'aloud',
 'alpha',
 'altar',
 'alter',
 'amass',
 'amaze',
 'amber',
 'amble',
 'amend',
 'amiss',
 'amity',
 'among',
 'ample',
 'amply',
 'amuse',
 'angel',
 'anger',
 'angle',
 'angry',
 'angst',
 'anime',
 'ankle',
 'annex',
 'annoy',
 'annul',
 'anode',
 'antic',
 'anvil',
 'aorta',
 'apart',
 'aphid',
 'aping',
 'apnea',
 'apple',
 'apply',
 'apron',
 'aptly',


In [18]:
# def f(A, first = False):
#     if len(A) == 1: return 1
#     if len(A) == 2: return 1.5
#     bestScore = 10000
#     for g in tqdm(sorted(G, key = lambda x: (x not in A, maxSizeSplit(x,A)))[:20], disable = not first):
#         t = 1
#         splits = getSplits(g, A, useWords = True)
#         if len(splits) == 1: continue
#         for res,split in splits.items():
#             t += len(split)/len(A) * f(split)
#         bestScore = min(bestScore, t)
#         if len(A) >= 7:
#             print(A,g,'-->',t)
#         if len(A) == 3 and (g in A and abs(t - 5/3) < 0.01):
# #             print("***exited early***")
#             return t
#         if len(A) == 4 and (g in A and abs(t - (1+2*3)/4) < 0.01):
#             print("***exited early***")
#             return t
#         if (g not in A) and abs(t - 2) < 0.01:
#             if len(A) >= 7: print("***exited early***")
#             return t
#     return bestScore

In [19]:
# f(C, first = True)
# f(['rover', 'rower', 'wooer'], first = True)

In [14]:
for k,v in sorted(getSplits('trace',S).items(), key = lambda x: x[1]):
    print(k,v)

12100 1
01112 1
11111 1
11210 1
11010 1
12210 1
12212 1
02110 1
12021 1
01122 1
01022 1
12101 1
11201 1
00121 1
02022 1
01221 1
11221 1
11011 1
11021 1
01220 1
01212 1
10220 1
11202 1
20221 1
21201 1
20202 1
21200 1
21020 1
22222 1
22202 1
20022 1
02021 1
11110 2
10112 2
02222 2
10012 2
02220 2
02111 2
12011 2
12010 2
10221 2
00120 2
12202 2
01021 2
20102 2
20110 2
20020 2
22220 2
22101 2
22100 2
22022 2
22020 2
02102 3
00221 3
01020 3
12002 3
10210 3
01120 3
02212 3
00122 3
12001 3
11002 3
00222 3
20101 3
21100 3
21002 3
20200 3
22001 3
00112 4
00212 4
12200 4
21101 4
20010 4
02020 5
10111 5
01111 5
01210 5
01012 5
02012 5
10021 5
22002 5
02210 6
22200 6
11200 7
02101 7
01202 7
10120 7
10011 7
21001 7
10110 8
00201 8
10010 8
02011 8
01102 9
01201 9
00021 9
11101 10
10102 10
02202 10
00111 10
22000 10
02010 11
20002 11
21000 11
00220 12
10202 12
11100 12
10201 12
10020 12
01110 13
02100 13
00011 13
12000 14
01011 14
20001 14
00022 15
00012 16
01200 17
00210 17
20000 17
10101 19
02001 1

In [6]:
import json

In [17]:
tree = json.load(open('combinedVals/mostParts,firstValid,maxSizeSplit,information1.json'))
# tree = tree['splits']['00100']
# tree = tree['splits']['00020']
# tree = tree['splits']['02000'] a
tree

{'guess': 'trace',
 'splits': {'00000': {'guess': 'soily',
   'splits': {'00000': {'guess': 'humph'},
    '00001': {'guess': 'nymph'},
    '00002': {'guess': 'dogma',
     'splits': {'00000': {'guess': 'funky',
       'splits': {'02002': {'guess': 'puppy'},
        '02202': {'guess': 'bunny'},
        '02222': {'guess': 'hunky'},
        '12002': {'guess': 'puffy'},
        '22002': {'guess': 'fuzzy'},
        '22202': {'guess': 'funny'}}},
      '00010': {'guess': 'jumpy'},
      '00020': {'guess': 'mummy'},
      '00100': {'guess': 'guppy'},
      '00120': {'guess': 'gummy'},
      '00200': {'guess': 'buggy'},
      '00220': {'guess': 'pygmy'},
      '10000': {'guess': 'buddy'},
      '10010': {'guess': 'muddy'},
      '10100': {'guess': 'pudgy'},
      '20010': {'guess': 'dumpy'},
      '20020': {'guess': 'dummy'}}},
    '00010': {'guess': 'flung',
     'splits': {'02200': {'guess': 'plumb',
       'splits': {'22220': {'guess': 'plump'}}},
      '02220': {'guess': 'plunk'},
      '1

In [18]:
def g(T, words):
#     r = [1]
#     if 'splits' in T:
#         for k in T['splits']:
#             r += [1 + x for x in g(T['splits'][k])]
#     return r
    r = []
    for w in words:
        s = 1
        curr = T
        res = check(curr['guess'],w)
        while res != '22222':
            s += 1
            curr = curr['splits'][res]
            res = check(curr['guess'],w)
            
        r.append(s)
#         print(w,s)
    return r
            
        

In [19]:
np.mean(g(tree,C)), len(g(tree,C)), max(g(tree,C))

(3.4304893893460373, 2309, 6)

In [15]:
g(tree,C), tree
# ['boxer', 'homer', 'hover', 'joker', 'mover', 'roger', 'rover']
# ['berry',
#  'demur',
#  'derby',
#  'every',
#  'femur',
#  'ferry',
#  'jerky',
#  'leery',
#  'lemur',
#  'merry',
#  'query',
#  'reedy']

([2,
  2,
  3,
  2,
  3,
  3,
  2,
  3,
  3,
  2,
  2,
  2,
  3,
  3,
  2,
  2,
  2,
  2,
  3,
  3,
  2,
  3,
  2,
  3,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  2,
  3,
  3,
  2,
  2,
  3,
  3,
  3],
 {'guess': 'shuln',
  'splits': {'00000': {'guess': 'foamy'},
   '00001': {'guess': 'piano'},
   '00002': {'guess': 'again'},
   '00010': {'guess': 'flail',
    'splits': {'00222': {'guess': 'avail'},
     '01200': {'guess': 'loamy'},
     '02200': {'guess': 'plaza'},
     '02201': {'guess': 'llama'},
     '02220': {'guess': 'plaid'},
     '22200': {'guess': 'flaky'}}},
   '00011': {'guess': 'bifid',
    'splits': {'00000': {'guess': 'plank'},
     '00002': {'guess': 'gland'},
     '00100': {'guess': 'flank'},
     '20000': {'guess': 'blank'},
     '20002': {'guess': 'bland'}}},
   '00012': {'guess': 'plain'},
   '00020': {'guess': 'koala'},
   '00100': {'guess': 'guava'},
   '00110': {'guess': 'quail'},
   '00120': {'guess': 'qualm'},
   '0200

In [5]:
%%time
get('tares', S)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.47 ms


225

In [31]:
from strategyTree import TreeGenerator

In [29]:
x = g(t)
np.mean(x), max(x)

(2.5, 4)

In [32]:
from itertools import permutations

In [33]:
allVals = [firstValid, maxSizeSplit, mostParts, information]
fns = sum([
    list(permutations(allVals, i)) for i in range(1,len(allVals)+1)
], [])
ans = []
for f in fns:
    t = TreeGenerator(vs = f)
    x = g(t.genStrategyTree(1, C))
    ans.append((t.stratName,np.mean(x), max(x)))

    Called with 113 candidates


  0%|          | 0/12947 [00:00<?, ?it/s]

[((0,), 'berry'), ((0,), 'bluer'), ((0,), 'boxer'), ((0,), 'buyer'), ((0,), 'defer')]
Submitting berry
    Called with 2 candidates
Got two candidates, guessing bluer
    Called with 1 candidates
Only one candidate boxer
    Called with 1 candidates
Only one candidate buyer
    Called with 17 candidates


  0%|          | 0/12947 [00:00<?, ?it/s]

[((0,), 'defer'), ((0,), 'demur'), ((0,), 'femur'), ((0,), 'fever'), ((0,), 'fewer')]
['defer', 'demur', 'femur', 'fever', 'fewer', 'lemur', 'leper', 'lever', 'never', 'newer', 'reign', 'renew', 'repel', 'resin', 'revel', 'sever', 'sewer']
{'22222': 1, '22002': 1, '02102': 1, '02122': 2, '02002': 1, '02022': 6, '02001': 2, '02021': 3}
(0,)
Submitting defer
    Called with 1 candidates
Only one candidate demur
    Called with 1 candidates
Only one candidate femur
    Called with 2 candidates
Got two candidates, guessing fever
    Called with 1 candidates
Only one candidate fewer
    Called with 1 candidates
Only one candidate lemur
    Called with 6 candidates


...

[((0.0, 0, 0, 0), 'never'), ((0.0, 0, 0, 0), 'newer'), ((0.0, 0, 0, 1), 'adaws'), ((0.0, 0, 0, 1), 'adown'), ((0.0, 0, 0, 1), 'advew')]
['never', 'newer', 'under']
{'22222': 1, '22022': 1, '10022': 1}
(0.0, 0, 0, 0)
Submitting never
    Called with 1 candidates
Only one candidate newer
    Called with 1 candidates
Only one candidate under
    Called with 1 candidates
Only one candidate peril
    Called with 10 candidates


  0%|          | 0/12947 [00:00<?, ?it/s]

[((1.3862943611198906, 1, 1, 1), 'serow'), ((1.3862943611198906, 1, 1, 1), 'shrow'), ((1.3862943611198906, 1, 1, 1), 'upbow'), ((1.3862943611198906, 1, 1, 1), 'whomp'), ((1.3862943611198906, 1, 1, 1), 'whoop')]
['poser', 'sever', 'sewer', 'sheer', 'shrew', 'sober', 'sower', 'super', 'surer', 'usher']
{'11110': 1, '22100': 1, '22101': 1, '21100': 2, '21202': 1, '21110': 1, '21111': 1, '21200': 1, '11100': 1}
(1.3862943611198906, 1, 1, 1)
Submitting serow
    Called with 1 candidates
Only one candidate poser
    Called with 1 candidates
Only one candidate sever
    Called with 1 candidates
Only one candidate sewer
    Called with 2 candidates
Got two candidates, guessing sheer
    Called with 1 candidates
Only one candidate super
    Called with 1 candidates
Only one candidate shrew
    Called with 1 candidates
Only one candidate sober
    Called with 1 candidates
Only one candidate sower
    Called with 1 candidates
Only one candidate surer
    Called with 1 candidates
Only one candidat

  0%|          | 0/12947 [00:00<?, ?it/s]

[((0.0, 0, 0, 0), 'serum'), ((0.0, 0, 0, 0), 'sperm'), ((0.0, 0, 0, 0), 'verso'), ((0.0, 0, 0, 1), 'abamp'), ((0.0, 0, 0, 1), 'abeam')]
['serum', 'sperm', 'verso']
{'22222': 1, '21102': 1, '12200': 1}
(0.0, 0, 0, 0)
Submitting serum
    Called with 1 candidates
Only one candidate sperm
    Called with 1 candidates
Only one candidate verso
    Called with 1 candidates
Only one candidate skier
    Called with 1 candidates
Only one candidate sneer
    Called with 1 candidates
Only one candidate weird


In [34]:
ans

[('firstValid', 3.5575221238938055, 7),
 ('maxSizeSplit', 3.0303030303030303, 4),
 ('mostParts', 2.9291338582677167, 5),
 ('information', 2.9076923076923076, 4),
 ('firstValid,maxSizeSplit', 3.15929203539823, 6),
 ('firstValid,mostParts', 3.265486725663717, 7),
 ('firstValid,information', 3.15929203539823, 6),
 ('maxSizeSplit,firstValid', 2.984, 4),
 ('maxSizeSplit,mostParts', 2.9612403100775193, 4),
 ('maxSizeSplit,information', 2.9612403100775193, 4),
 ('mostParts,firstValid', 2.901639344262295, 5),
 ('mostParts,maxSizeSplit', 2.9291338582677167, 5),
 ('mostParts,information', 2.9140625, 5),
 ('information,firstValid', 2.894308943089431, 4),
 ('information,maxSizeSplit', 2.9076923076923076, 4),
 ('information,mostParts', 2.9076923076923076, 4),
 ('firstValid,maxSizeSplit,mostParts', 3.150442477876106, 6),
 ('firstValid,maxSizeSplit,information', 3.150442477876106, 6),
 ('firstValid,mostParts,maxSizeSplit', 3.15929203539823, 6),
 ('firstValid,mostParts,information', 3.15929203539823, 

In [35]:
ans.sort(key = lambda x: list(x)[1:])

In [36]:
ans

[('information,firstValid', 2.894308943089431, 4),
 ('information,firstValid,maxSizeSplit', 2.894308943089431, 4),
 ('information,firstValid,mostParts', 2.894308943089431, 4),
 ('information,maxSizeSplit,firstValid', 2.894308943089431, 4),
 ('information,mostParts,firstValid', 2.894308943089431, 4),
 ('information,firstValid,maxSizeSplit,mostParts', 2.894308943089431, 4),
 ('information,firstValid,mostParts,maxSizeSplit', 2.894308943089431, 4),
 ('information,maxSizeSplit,firstValid,mostParts', 2.894308943089431, 4),
 ('information,maxSizeSplit,mostParts,firstValid', 2.894308943089431, 4),
 ('information,mostParts,firstValid,maxSizeSplit', 2.894308943089431, 4),
 ('information,mostParts,maxSizeSplit,firstValid', 2.894308943089431, 4),
 ('mostParts,firstValid,maxSizeSplit,information', 2.894308943089431, 5),
 ('mostParts,maxSizeSplit,firstValid,information', 2.894308943089431, 5),
 ('mostParts,maxSizeSplit,information,firstValid', 2.894308943089431, 5),
 ('mostParts,firstValid', 2.90163

In [216]:
class Solver(object):
    def __init__(self, words):
        self.words = words
        self.memo = {}
        self.best = {}
    
    def encode(self, sub):
        t = 0
        for w in sub:
            t |= 1 << (len(self.words) - self.words.index(w) - 1)
        return t
    
    def solve(self):
        self.h(self.words)
        return self.memo[self.encode(self.words)]
        
    def h(self, sub, depth = 0):
        c = self.encode(sub)
        if c in self.memo: 
            return self.memo[c]
        
        ans = 1
        if len(sub) == 1:
            self.best[c] = sub[0]
            ans = 1
        elif len(sub) == 2:
            self.best[c] = sub[0]
            ans = 1.5
        else:
            bestScore = -1
            bestGuess = ""
            for g in tqdm(G, disable = depth >= 2):
                t = 1
                splits = getSplits(g, sub, useWords = True)
                if len(splits) == 1: continue
                for res, split in splits.items():
                    if res == '22222': continue
                    t += len(split)/len(sub) * (self.h(split, depth + 1))
#                     if first:
#                         print(split, t, self.h(split))
                
                if bestScore == -1 or t < bestScore:
                    bestScore = t
                    bestGuess = g
                    
#                 print(g,'final ',t, bestScore, getSplits(g, sub, useWords = True))
                
            ans = bestScore
            self.best[c] = bestGuess
        self.memo[c] = ans 
        return ans
    
    def genTree(self, sub = None):
        if sub is None:
            sub = self.words
#         print(self.memo[self.encode(sub)])
        guess = self.best[(self.encode(sub))]
        tree = {
            'guess': guess
        }
        if len(sub) != 1:
            tree['splits'] = {}
            splits = getSplits(guess, sub, useWords = True)
            for res, split in splits.items():
                if res == '22222': continue
                tree['splits'][res] = self.genTree(split)
        
        return tree
        

In [252]:
s = Solver(words = C)
# s = Solver(words = ['boxer', 'homer', 'hover', 'joker', 'mover', 'roger', 'rover'])
# s = Solver(words = ['other','blood','tares'])
# bin(s.encode(s.words))
s.solve()

  0%|          | 0/12947 [00:00<?, ?it/s]

  0%|          | 0/12947 [00:00<?, ?it/s]

  0%|          | 0/12947 [00:00<?, ?it/s]

  0%|          | 0/12947 [00:00<?, ?it/s]

  0%|          | 0/12947 [00:00<?, ?it/s]

...

  0%|          | 0/12947 [00:00<?, ?it/s]

  0%|          | 0/12947 [00:00<?, ?it/s]

  0%|          | 0/12947 [00:00<?, ?it/s]

  0%|          | 0/12947 [00:00<?, ?it/s]

2.0769230769230766

In [180]:
for k,v in s.best.items():
    print(bin(k),'-->',v, s.memo[k])

0b1000000 --> boxer 1
0b1000 --> joker 1
0b101 --> mover 1.5
0b1001 --> joker 1.5
0b100 --> mover 1
0b1100 --> joker 1.5
0b1 --> rover 1
0b1101 --> mover 1.6666666666666665
0b1000001 --> boxer 1.5
0b1000100 --> boxer 1.5
0b1000101 --> mover 1.6666666666666665
0b1001000 --> boxer 1.5
0b1001001 --> aback 1.9999999999999998
0b1001100 --> aback 1.9999999999999998
0b1001101 --> jambe 2.0
0b10 --> roger 1
0b11 --> roger 1.5
0b111 --> mover 1.6666666666666665
0b1010 --> joker 1.5
0b1011 --> roger 1.6666666666666665
0b1111 --> rover 1.75
0b1000010 --> boxer 1.5
0b1000011 --> roger 1.6666666666666665
0b1000111 --> rover 1.75
0b1001010 --> aback 1.9999999999999998
0b1001011 --> rejig 2.0
0b1001111 --> rover 2.0
0b110000 --> homer 1.5
0b100100 --> homer 1.5
0b10000 --> hover 1
0b10001 --> hover 1.5
0b10011 --> hover 1.6666666666666665
0b100000 --> homer 1
0b100010 --> homer 1.5
0b10100 --> hover 1.5
0b10101 --> abmho 1.9999999999999998
0b110100 --> homer 1.6666666666666665
0b110101 --> homer 1.75

In [189]:
for k,v in getSplits('defer',['berry',
 'demur',
 'derby',
 'every',
 'femur',
 'ferry',
 'jerky',
 'leery',
 'lemur',
 'merry',
 'query',
 'reedy'],useWords=True).items():
    print(k,v,bin(s.encode(v)))

02001 ['berry', 'jerky', 'merry'] 0b100000100100
22002 ['demur'] 0b10000000000
22001 ['derby'] 0b1000000000
01011 ['every'] 0b100000000
02102 ['femur'] 0b10000000
02101 ['ferry'] 0b1000000
02011 ['leery'] 0b10000
02002 ['lemur'] 0b1000
01001 ['query'] 0b10
12011 ['reedy'] 0b1


In [141]:
(9*2 + 3*3)/12

2.25

In [185]:
(10*2 + 2*3)/12

2.1666666666666665

In [149]:
for k,v in getSplits('other',['other','blood','tares'],useWords=True).items():
    print(k,v,bin(s.encode(v)))

22222 ['other'] 0b100
10000 ['blood'] 0b10
01021 ['tares'] 0b1


In [22]:
(1 + 7 *2 + 4 * 3)/12

2.25

In [237]:
s.genTree()

{'guess': 'coley',
 'splits': {'10020': {'guess': 'bicep'},
  '21210': {'guess': 'cello'},
  '20020': {'guess': 'cheek', 'splits': {'22020': {'guess': 'chief'}}},
  '20010': {'guess': 'chess'},
  '20120': {'guess': 'clued'},
  '22020': {'guess': 'coven'},
  '22022': {'guess': 'covey'},
  '11012': {'guess': 'decoy'},
  '10022': {'guess': 'dicey'},
  '10120': {'guess': 'excel'},
  '11010': {'guess': 'gecko'},
  '10010': {'guess': 'medic'}}}

In [225]:
len(s.memo), 2**len(s.words) - 1

(14560, 524287)

In [238]:
tree

{'guess': 'coley',
 'splits': {'10010': {'guess': 'medic'},
  '10020': {'guess': 'bicep'},
  '10022': {'guess': 'dicey'},
  '10120': {'guess': 'excel'},
  '11010': {'guess': 'gecko'},
  '11012': {'guess': 'decoy'},
  '20010': {'guess': 'chess'},
  '20020': {'guess': 'cheek', 'splits': {'22020': {'guess': 'chief'}}},
  '20120': {'guess': 'clued'},
  '21210': {'guess': 'cello'},
  '22020': {'guess': 'coven'},
  '22022': {'guess': 'covey'}}}

In [6]:
import originalWordLists
from utils import check
import json

In [4]:
with open('originalBestTree.json','r') as f:
    tree = json.load(f)

In [10]:
with open('originalBestTreeList.txt','w') as g:
    for s in originalWordLists.answers:
        ans = []
        curr = tree
        while 1:
            ans.append(curr['guess'])
            if curr['guess'] == s: break
            res = check(curr['guess'], s)
            curr = curr['splits'][res]
        g.write(','.join(ans) + '\n')

In [22]:
df = pd.DataFrame()
df['guess'] = G
df['score'] = [len(getSplits(g, S)) for g in tqdm(G)]
df

  0%|          | 0/12947 [00:00<?, ?it/s]

,guess,score
0,aahed,68
1,aalii,58
2,aargh,75
3,aarti,82
4,abaca,38
...,...,...
12942,zuzim,33
12943,zygal,62
12944,zygon,60
12945,zymes,49


In [32]:
df['in'] = [int(g in S) for g in G]

In [45]:
df = df.sort_values(by=['score','in'], ascending=[False, False]).reset_index(drop=True)

In [41]:
df[df['guess'] == 'salet']

,guess,score,in
2309,salet,148,0


In [46]:
df

,guess,score,in
0,trace,150,1
1,crate,148,1
2,salet,148,0
3,reast,147,0
4,parse,146,1
...,...,...,...
12942,yukky,27,0
12943,ayaya,25,0
12944,pzazz,25,0
12945,jujus,23,0
